In [ ]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D,Activation, SpatialDropout2D
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D
from keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras import regularizers
import tensorflow as tf
import numpy as np

In [ ]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10

In [ ]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
imdgen = ImageDataGenerator(
    featurewise_center = False,  # set input mean to 0 over the dataset
    samplewise_center = False,  # set each sample mean to 0
    featurewise_std_normalization = False,  # divide inputs by std of the dataset
    samplewise_std_normalization = False,  # divide each input by its std
    zca_whitening = False,  # apply ZCA whitening
    rotation_range = 15,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range = 0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range = 0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip = True,  # randomly flip images
    vertical_flip = False,  # randomly flip images
    shear_range=0.2,
    zoom_range=0.2,
)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
imdgen.fit(x_train)

# fit the model on the batches generated by datagen.flow()
dgen = imdgen.flow(x_train, y_train, batch_size=128)

In [ ]:
# For reproducibility
np.random.seed(1000)

# Create the model
model = Sequential()
model.add(Conv2D(16, (3,3), strides=(1,1), padding='same', use_bias=False, input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))

for i in range(0,3):
    model.add(Conv2D(32*(2**i), (3,3), strides=(1,1), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3), strides=(1,1), padding='same', use_bias=False))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='same'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))
checkpoint = ModelCheckpoint("CIFAR_10_YOLO_with_dropout.h5", monitor='val_loss', verbose=2, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        432       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 16)       64        
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        4

In [ ]:
# Train the model
model.fit(x_train, y_train,batch_size=128,epochs=10,validation_data=(x_test, y_test),callbacks=callbacks_list,shuffle=True)

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 1.8974 - categorical_accuracy: 0.3333
Epoch 1: val_loss improved from inf to 1.89173, saving model to CIFAR_10_YOLO_with_dropout.h5
391/391 [==============================] - 179s 454ms/step - loss: 1.8974 - categorical_accuracy: 0.3333 - val_loss: 1.8917 - val_categorical_accuracy: 0.3810
Epoch 2/10
391/391 [==============================] - ETA: 0s - loss: 1.5687 - categorical_accuracy: 0.4325
Epoch 2: val_loss did not improve from 1.89173
391/391 [==============================] - 169s 431ms/step - loss: 1.5687 - categorical_accuracy: 0.4325 - val_loss: 1.8951 - val_categorical_accuracy: 0.3471
Epoch 3/10
391/391 [==============================] - ETA: 0s - loss: 1.4204 - categorical_accuracy: 0.4871
Epoch 3: val_loss did not improve from 1.89173
391/391 [==============================] - 169s 432ms/step - loss: 1.4204 - categorical_accuracy: 0.4871 - val_loss: 1.9338 - val_categorical_accuracy: 0.3560
Epoch 4/10


In [ ]:
train_mse = model.evaluate(x_train, y_train, verbose=0)
test_mse = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
print('Train loss : %.3f %% , Test loss : %.3f %%' % (train_mse[0]*100, test_mse[0]*100))
print('Train accuracy : %.3f %% , Test accuracy : %.3f %%' % (train_mse[1]*100, test_mse[1]*100))

Train loss : 93.024 % , Test loss : 98.835 %
Train accuracy : 66.686 % , Test accuracy : 64.770 %
